In [19]:
from jupyter_plotly_dash import JupyterDash
import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State
import base64

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps

#Imports the CRUD Python module, AKA, the Animal_Shelter.py file
from Animal_Shelter import AnimalShelter


###########################
# Data Manipulation / Model
###########################
#Username used in Database authentication
username = "aacuser"
#Password used in Database authentication
password = "Chocolate3"
#Animal_Shelter class object instantion
shelter = AnimalShelter(username, password)

#df is equal to the data being loaded in from the AAC database
df = pd.DataFrame.from_records(shelter.read({}))



#########################
# Dashboard Layout / View
#########################
#Creates the app which will hold the UI
app = JupyterDash('SimpleExample')

#Loads the Grazioso Salvare logo for later use
image_filename = 'Grazioso Salvare Logo.png' 
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#Start of the app UI layout which uses html to develop the web UI
app.layout = html.Div([
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))), #Creates a header text in the middle-top of the UI
    html.Center(html.H2('Developer: Dominic Clapper')), #Creates a smaller header text beneath the SNHU CS-340 Dashboard text
    html.Hr(),
    #Displays the Grazioso Salvare logo in the middle of the UI, beneath the two headers
    html.Center(html.Img(
        src='data:image/png;base64,{}'.format(encoded_image.decode()),
        style={'height':'20%','width':'20%'}
        )),
    html.Div(
        #Creates a set of buttons that can be clicked by the user to filter what animals they are searching for
        dcc.RadioItems(
            id = 'filter-type', #Defines the filter for how the animals should be searched from the database
            #A list that holds all of the different filter types: Water Rescue, Mountain/Wilderness Rescue, Disaster/Individual Tracking, and Reset
            options = [
                {'label': 'Water Rescue', 'value': 'water'},
                {'label': 'Mountain/Wilderness Rescue', 'value': 'mount/wild'},
                {'label': 'Disaster Rescue/Indiv. Tracking', 'value': 'disaster/indiv'},
                {'label': 'Reset', 'value': 'reset'}
            ]    
        )
    ),
    html.Hr(),
    #Start of the datatable that is displayed in the UI
    dt.DataTable(
        #Represents the unique ID of the table for later use
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        #Loads the database stored in df to the table
        data=df.to_dict('records'),
        #Makes the table uneditable
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        #Makes it impossible to select columns
        column_selectable=False,
        #Allows for only one row to be selected at a time
        row_selectable="single",
        row_deletable=False,
        #List of columns selected
        selected_columns=[],
        #List of rows selected
        selected_rows=[],
        page_action="native",
        #Sets the table to the first page of the database
        page_current= 0,
        #Sets the page size to 10. Meaning only 10 dogs will appear per page
        page_size= 10,
    ),
    html.Br(),
    html.Hr(),
    
    #Holds the map and pie chart and forces them to be next to one another in the UI
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
                html.Div([
                    #Creates a drop down list of filter options for the pie chart such as Age and Breed
                    #It will alter the pie chart based on the selected filter
                    dcc.Dropdown(
                        id='pie_dropdown',
                        className='col s12 m6',
                        #Defines the potential filter options for the dropdown list for the pie chart
                        options=[
                            {'label': 'Breed', 'value': 'breed'},
                            {'label': 'Age', 'value': 'age_upon_outcome'}
                        ],
                        #The base value is breed for the filter type
                        value='breed',
                        #Makes it so only one option can be selected at a time
                        multi=False,
                        clearable=False
                        )
                    ],
                style={"width":"20%"}
                ),
                #Displays the Pie Chart representing the dogs with the filter options from the radio item object in mind
                html.Div(
                    id='graph-id',
                    className='col s12 m6',
                    style={"width":"45%"}
                    
                ),
                #Displays the Geolocation Map of where dogs are in terms of location. A user will have to click on the dog in the table in order to show it's location on the map
                html.Div(
                    id='map-id',
                    className='col s12 m6',
                    style={"width":"45%"}
                    )
                ])
])

#############################################
# Interaction Between Components / Controller
#############################################

#Callback command that works whenever a filter option is selected. When a user selects the bubble in the radio items object, it changes the filter option.
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns'),
               Output('datatable-id','selected_rows')],
              [Input('filter-type', 'value')])

def update_dashboard(filter_type):
        #If the filter is set to water rescues, then the dataFrame will load only dogs with the specified info. The info is based off of the chart provided by Grazioso Salvare
        if filter_type == 'water':
            df = pd.DataFrame.from_records(shelter.read({'animal_type': 'Dog', 
                                                         'breed': {'$in': ['Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland']},
                                                         'sex_upon_outcome': 'Intact Female',
                                                         'age_upon_outcome_in_weeks': {'$gte':26.0},
                                                         'age_upon_outcome_in_weeks': {'$lte': 156.0}
            }))
        #If the filter is set to mountain/wild rescues, then the dataFrame will load only dogs with the specified info. The info is based off of the chart provided by Grazioso Salvare
        elif filter_type == 'mount/wild':
            df = pd.DataFrame.from_records(shelter.read({'animal_type': 'Dog', 
                                                         'breed': {'$in': ['German Shepherd', 'Alaskan Malamute', 'Old English Sheepdog', 'Siberian Husky', 'Rottweiler']},
                                                         'sex_upon_outcome': 'Intact Male',
                                                         'age_upon_outcome_in_weeks': {'$gte':26.0},
                                                         'age_upon_outcome_in_weeks': {'$lte': 156.0}
            }))
        #If the filter is set to disasters/individual tracking, then the dataFrame will load only dogs with the specified info. The info is based off of the chart provided by Grazioso Salvare
        elif filter_type == 'disaster/indiv':
            df = pd.DataFrame.from_records(shelter.read({'animal_type': 'Dog', 
                                                         'breed': {'$in': ['Doberman Pinscher', 'German Shepherd', 'Golden Retriever', 'Bloodhound', 'Rottweiler']},
                                                         'sex_upon_outcome': 'Intact Male',
                                                         'age_upon_outcome_in_weeks': {'$gte':20.0},
                                                         'age_upon_outcome_in_weeks': {'$lte': 300.0}
            }))
        elif filter_type == 'reset':
            df = pd.DataFrame.from_records(shelter.read({}))
        #If the filter is set to none of the following, or reset, then the dataFrame will load all dogs. The info is based off of the chart provided by Grazioso Salvare
        else:
            df = pd.DataFrame.from_records(shelter.read({}))
            
        data=df.to_dict('records')
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        
        selected_rows = [0]
        
        #Returns the data found in the database after a filtered query is run on the database. It also returns the list for selected rows starting at row 0/the first row
        return (data, columns, selected_rows)



#Callback command that works when a row is selected in the table. Highlights the row selected when the circle is pressed at the start of a row
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_rows')]
)

#Works with the Callback command above to highlight the row selected
def update_styles(selectedRows):
    return [{
        'if': { 'row_index': i },
        'background_color': '#D2F3FF'
    } for i in selectedRows]


#Callback command that changes the results shown on the pie chart when a filter is used on the table or in the dropdown list next to the chart. If a user selects the Water filter, then the pie chart will change
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data"),
     Input('pie_dropdown', 'value')])

#Works with the Callback command above to change the results of the pie chart whenever a filter is used
def update_graphs(viewData, dropdownValue):
    dff = pd.DataFrame.from_dict(viewData)
    return [
        dcc.Graph(            
            figure = px.pie(
                data_frame=dff, 
                names=dropdownValue)
        )    
    ]

#Callback command that changes the results shown in the map whenever a new selection is made in the table. If a user selects a row in the table, then the map will update
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data"),
     Input('datatable-id', 'selected_rows'),])

#Update Map function that works with the Callback command right above it to update the map when necessary
def update_map(viewData, selectedRows):   
    
    #If no row is selected, then the first row is shown in the map
    if not selectedRows:
        selected_row = 0
    #If a row is selected, then that row will be shown in the map
    else:   
        selected_row = selectedRows[0]
       
    #dff stores the data from the table                           
    dff = pd.DataFrame.from_dict(viewData)   
    
    posX = dff.iloc[selected_row, 13]
    posY = dff.iloc[selected_row, 14]
    
    #Retrieves the name of the animal from the specified row and assigns it to name. If the name does not exist, then No Name is set as the name value
    name = dff.iloc[selected_row,9] 
    if name == "":
        name = "No Name"
        
    #Returns the map object with a marker on it displaying the location of the dog via a tooltip.
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[posX,posY], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[posX,posY], children=[
                dl.Tooltip(dff.iloc[selected_row,4]),
                dl.Popup([
                    html.P("Animal Name"),
                    html.H1(name)
                ])
            ])
        ])
    ]                           


#Calls the app variable that was made earlier to store the UI
app